In [ ]:
# ercot_volatility_analysis.py (Plotly version, cleaned + EGARCH with t-dist + trimming + prep for GARCH-X)

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from arch import arch_model
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt  # Only for ACF/PACF as Plotly doesn't support these directly

# === 1. Load combined dataset =====
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
df = pd.read_csv(file_path, parse_dates=['datetime'])

# === 2. Select zone and calculate log returns ===
zone = 'LZ_HOUSTON'
price_col = f'{zone}_rtm'
df = df[['datetime', price_col, 'wind_mw', 'solar_mw', 'net_output_mw']].copy()
df = df[df[price_col] > 0]  # Filter out non-positive prices

# Calculate log returns
df['log_return'] = np.log(df[price_col]).diff() * 100

# Drop rows with NaN or inf in log_return
df['log_return'] = df['log_return'].replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=['log_return'])

# === Trim extreme outliers in log return ===
outlier_threshold = 300
df['log_return'] = df['log_return'].clip(lower=-outlier_threshold, upper=outlier_threshold)

# === 3. Plot distribution of trimmed returns ===
fig = px.histogram(df, x='log_return', nbins=100, marginal='rug',
                   title=f'Trimmed Log Return Distribution – {zone}',
                   labels={'log_return': 'Log Return (%)'})
fig.update_layout(template='plotly_white')
fig.show()

# === 4. ADF Test ===
adf_result = adfuller(df['log_return'])
print(f"ADF Statistic: {adf_result[0]:.4f}")
print(f"p-value: {adf_result[1]:.4f}")

# === 5. ACF/PACF Plots (Matplotlib) ===
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
plot_acf(df['log_return'], lags=40, ax=ax[0])
plot_pacf(df['log_return'], lags=40, ax=ax[1])
ax[0].set_title('ACF of Log Returns')
ax[1].set_title('PACF of Log Returns')
plt.tight_layout()
plt.show()

# === 6. Fit GARCH(1,1) model ===
log_returns_clean = df['log_return'].dropna()
model_garch = arch_model(log_returns_clean, vol='Garch', p=1, q=1)
res_garch = model_garch.fit(disp='off')
df = df.loc[log_returns_clean.index].copy()
df['volatility_garch'] = res_garch.conditional_volatility.values

print("\nGARCH(1,1) Parameters:")
print(res_garch.summary())

# === 7. Fit EGARCH(1,1) with t-distribution ===
model_egarch = arch_model(log_returns_clean, vol='EGarch', p=1, q=1, dist='t')
res_egarch = model_egarch.fit(disp='off')
df['volatility_egarch'] = res_egarch.conditional_volatility.values

print("\nEGARCH(1,1) Parameters (t-distribution):")
print(res_egarch.summary())

# === 8. Plot both volatilities ===
fig = px.line(df, x='datetime', y=['volatility_garch', 'volatility_egarch'],
              title=f'GARCH vs EGARCH (t-dist) Conditional Volatility – {zone}',
              labels={'value': 'Volatility (%)', 'datetime': 'Date', 'variable': 'Model'})
fig.update_layout(template='plotly_white')
fig.show()

# === 9. Segment and compare volatility (before/after 2024) ===
df['period'] = np.where(df['datetime'] < '2024-01-01', 'pre_BESS', 'post_BESS')
mean_vols = df.groupby('period')[['volatility_garch', 'volatility_egarch']].mean()
print("\nAverage Volatility by Period:")
print(mean_vols)

# === 10. Scatter matrix: volatility vs renewables/BESS ===
fig = px.scatter_matrix(df.dropna(subset=['volatility_egarch', 'wind_mw', 'solar_mw', 'net_output_mw']),
                        dimensions=['volatility_egarch', 'wind_mw', 'solar_mw', 'net_output_mw'],
                        title='EGARCH Volatility vs Renewables and BESS')
fig.update_layout(template='plotly_white', height=700)
fig.show()

# === Ready for future: GARCH-X (with exogenous variables) ===
# Example:
# model_garchx = arch_model(log_returns_clean, vol='Garch', p=1, q=1, x=df[['wind_mw', 'solar_mw', 'net_output_mw']].shift(1))


In [ ]:
# ercot_volatility_analysis.py

import pandas as pd
import numpy as np
import plotly.express as px
from arch import arch_model
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# === 1. Load data ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
df = pd.read_csv(file_path, parse_dates=['datetime'])

# === 2. Preprocessing ===
zone = 'LZ_HOUSTON'
price_col = f'{zone}_rtm'
df = df[['datetime', price_col, 'wind_mw', 'solar_mw', 'net_output_mw']].copy()
df = df[df[price_col] > 0]
df['log_return'] = np.log(df[price_col]).diff() * 100
df['log_return'] = df['log_return'].replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=['log_return'])
df['log_return'] = df['log_return'].clip(-300, 300)

# === 3. ADF Test ===
adf_stat, p_value, *_ = adfuller(df['log_return'])
print(f"ADF Statistic: {adf_stat:.4f}, p-value: {p_value:.4f}")

# === 4. ACF/PACF ===
fig, ax = plt.subplots(2, 1, figsize=(12, 6))
plot_acf(df['log_return'], lags=40, ax=ax[0])
plot_pacf(df['log_return'], lags=40, ax=ax[1])
ax[0].set_title('ACF of Log Returns')
ax[1].set_title('PACF of Log Returns')
plt.tight_layout()
plt.show()

# === 5. EGARCH Fit ===
log_returns_clean = df['log_return']
model_egarch = arch_model(log_returns_clean, vol='EGarch', p=1, q=1, dist='t')
res_egarch = model_egarch.fit(disp='off')
df['volatility_egarch'] = res_egarch.conditional_volatility.values
print(res_egarch.summary())

# === 6. Time series plot ===
fig = px.line(df, x='datetime', y='volatility_egarch', title='EGARCH Volatility Over Time')
fig.update_layout(template='plotly_white')
fig.show()

# === 7. Boxplot Pre vs Post BESS ===
df['period'] = np.where(df['datetime'] < '2024-01-01', 'Pre-BESS', 'Post-BESS')
fig = px.box(df, x='period', y='volatility_egarch', title='EGARCH Volatility Pre vs Post BESS')
fig.update_layout(template='plotly_white')
fig.show()

# === 8. Scatter Matrix ===
fig = px.scatter_matrix(df.dropna(subset=['volatility_egarch', 'wind_mw', 'solar_mw', 'net_output_mw']),
                        dimensions=['volatility_egarch', 'wind_mw', 'solar_mw', 'net_output_mw'],
                        title='EGARCH Volatility vs Renewables and BESS')
fig.update_layout(template='plotly_white', height=700)
fig.show()


In [ ]:
import pandas as pd
import numpy as np
from arch import arch_model
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# === Load data ===
df = pd.read_csv(r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv", parse_dates=['datetime'])

# === Define zones to analyze ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']

# === Results containers ===
summary_list = []
volatility_df = pd.DataFrame({'datetime': df['datetime']})

# === Loop over zones ===
for zone in zones:
    price_col = f'{zone}_rtm'
    if price_col not in df.columns:
        print(f"❌ Missing column: {price_col}, skipping...")
        continue

    temp = df[['datetime', price_col]].copy()
    temp = temp[temp[price_col] > 0].dropna()
    temp['log_return'] = np.log(temp[price_col]).diff() * 100
    temp['log_return'] = temp['log_return'].replace([np.inf, -np.inf], np.nan).clip(-300, 300)
    temp = temp.dropna(subset=['log_return'])

    # === Fit EGARCH(1,1) with t-distribution ===
    model = arch_model(temp['log_return'], vol='EGarch', p=1, q=1, dist='t')
    res = model.fit(disp='off')

    vol_col = f"{zone}_vol_egarch"
    volatility_df = volatility_df.merge(temp[['datetime']].assign(**{vol_col: res.conditional_volatility.values}), on='datetime', how='left')

    # === Collect summary ===
    summary_list.append({
        'Zone': zone,
        'LogLik': round(res.loglikelihood, 2),
        'AIC': round(res.aic, 1),
        'ν (t-dist)': round(res.params.get('nu', np.nan), 2),
        'ω': round(res.params.get('omega', np.nan), 4),
        'α1': round(res.params.get('alpha[1]', np.nan), 4),
        'β1': round(res.params.get('beta[1]', np.nan), 4),
        'mu': round(res.params.get('mu', np.nan), 4)
    })

# === Create summary table ===
summary_df = pd.DataFrame(summary_list)
print("\n📊 EGARCH Model Summary Per Zone:")
print(summary_df.to_string(index=False))

# === Plot 4 volatility curves as subplots ===
fig = make_subplots(rows=2, cols=2, shared_xaxes=True, subplot_titles=zones)

positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

for i, zone in enumerate(zones):
    vol_col = f"{zone}_vol_egarch"
    if vol_col not in volatility_df.columns:
        continue
    row, col = positions[i]
    fig.add_trace(go.Scatter(x=volatility_df['datetime'], y=volatility_df[vol_col],
                             mode='lines', name=zone,
                             line=dict(width=1)), row=row, col=col)

fig.update_layout(title='EGARCH Conditional Volatility by Zone',
                  template='plotly_white', height=700, showlegend=False)
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Volatility (%)')
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define cutoff
cutoff = pd.to_datetime("2024-01-01")

# Smoothing: 1-month rolling average (hourly data = 24*30 = 720)
window_hours = 720
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']
vol_df = volatility_df.copy()

# Create smoothed series and flag pre/post-BESS
for zone in zones:
    col = f"{zone}_vol_egarch"
    vol_df[f"{col}_smooth"] = vol_df[col].rolling(window=window_hours, min_periods=1).mean()
vol_df['period'] = np.where(vol_df['datetime'] < cutoff, 'Pre-BESS', 'Post-BESS')

# Plotting
fig = make_subplots(rows=2, cols=2, subplot_titles=zones, shared_xaxes=True, shared_yaxes=True)

positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

for i, zone in enumerate(zones):
    row, col = positions[i]
    col_smooth = f"{zone}_vol_egarch_smooth"
    
    pre = vol_df[vol_df['datetime'] < cutoff]
    post = vol_df[vol_df['datetime'] >= cutoff]
    
    # Add pre-BESS smoothed line
    fig.add_trace(go.Scatter(
        x=pre['datetime'], y=pre[col_smooth],
        mode='lines', name=f"{zone} Pre-BESS",
        line=dict(color='gray'), showlegend=False
    ), row=row, col=col)

    # Add post-BESS smoothed line
    fig.add_trace(go.Scatter(
        x=post['datetime'], y=post[col_smooth],
        mode='lines', name=f"{zone} Post-BESS",
        line=dict(color='steelblue'), showlegend=False
    ), row=row, col=col)

fig.update_layout(
    height=700, width=1100,
    title_text="Rolling EGARCH Volatility by Zone (Pre/Post BESS)",
    template="plotly_white"
)
fig.update_yaxes(title_text="Volatility (%)")
fig.update_xaxes(title_text="Date")
fig.update_xaxes(showticklabels=True, tickformat="%Y")  # Show year labels
fig.update_layout(xaxis_showticklabels=True)
fig.show()
